In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-io

In [6]:
import zipfile
from PIL import Image
import io
import tensorflow as tf
import tensorflow.keras.applications
import os
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB7
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.applications import EfficientNetB5


In [8]:
folder_path = '/kaggle/input/wikiart-monet-renoit/claude-monet'
X = []
monet = 0

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        monet += 1
        image = Image.open(file_path)
        image = image.resize((456, 456))
        image_array = np.array(image)
        X.append(image_array)

folder_path = '/kaggle/input/wikiart-monet-renoit/pierre-auguste-renoir'
renoir = 0

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        renoir += 1
        image = Image.open(file_path)
        image = image.resize((456, 456))
        image_array = np.array(image)
        X.append(image_array)

X = np.array(X)

In [4]:
base_model = EfficientNetB5(weights='imagenet')


123465288/123465288 [==============================] - 6s 0us/step


In [5]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(2, activation='softmax')
])

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
y = np.array([(0, 1) for x in range(monet)]+ [(1, 0) for x in range(renoir)])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.5, random_state=42
)

In [11]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=1)

ResourceExhaustedError: ignored

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
# Extract the training metrics from the history object
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']

# Extract the validation metrics from the history object (if applicable)
val_loss = history.history.get('val_loss')
val_accuracy = history.history.get('val_accuracy')

# Plot the training metrics
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Train Loss')
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Train Accuracy')

# Plot the validation metrics if available
if val_loss is not None:
    plt.plot(range(1, len(val_loss) + 1), val_loss, label='Val Loss')
if val_accuracy is not None:
    plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Val Accuracy')

# Set the plot title and labels
plt.title('Training Metrics')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()

# Show the plot
plt.show()